In [1]:
#Import Required libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [2]:
finviz_url = "https://finviz.com/quote.ashx?t="
ticker = "TSLA"
url = finviz_url + ticker

In [3]:
req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'})
response = urlopen(req)

news_table = {}
html = BeautifulSoup(response)
news_table = html.find(id='news-table')
# news_table

dataRows = news_table.findAll('tr')


In [4]:
dataRows

[<tr><td align="right" style="white-space:nowrap" width="130">Mar-26-21 03:53PM  </td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://www.investors.com/news/nio-stock-buy-now/?src=A00220" target="_blank">Is Nio Stock A Buy? Chip Shortage, EV Competition Weigh On Outlook</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> Investor's Business Daily</span></div></div></td></tr>,
 <tr><td align="right" width="130">03:21PM  </td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/news/column-feds-rule-musk-tesla-192125082.html" target="_blank">Column: Feds rule that Musk and Tesla are no friend to workers</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> LA Times</span></div></div></td></tr>,
 <tr><td align="right" width="130">03:08PM  </td><td align="left"><div class="news-link-con

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(columns=['News_Title', 'Time'])

In [7]:
for i, table_row in enumerate(dataRows):
    a_text = table_row.a.text
    td_text = table_row.td.text
    
    df = df.append({'News_Title': a_text, 'Time': td_text}, ignore_index=True)

In [8]:
df

,News_Title,Time
0,"Is Nio Stock A Buy? Chip Shortage, EV Competit...",Mar-26-21 03:53PM
1,Column: Feds rule that Musk and Tesla are no f...,03:21PM
2,How the pandemic has led to a boom in used car...,03:08PM
3,"Dow, Tech Stocks Rally As Treasury Yields Jump...",01:19PM
4,Tesla's Touchscreen-Based Gear Selection Doesn...,01:11PM
...,...,...
95,Elon Musk Confirms Refreshed Tesla Model S Wil...,05:53PM
96,Tesla Supplier AU Optronics In Talks To Manufa...,03:48PM
97,Tesla's in-car cameras raise privacy concerns:...,02:39PM
98,"Dow, Tech Stocks Fall As Powell Calls U.S. Eco...",12:04PM


In [9]:
#Import Dependencies for sentiment analysis using our neural network
import tensorflow as tf
import numpy
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [10]:
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
model = tf.keras.models.load_model('./model1.h5')

In [11]:
oov_tok = '<OOV>'
trunc_type = 'post'
padding_type='post'
vocab_size =1000
max_length = 142

def preprocessText(text):
    sequences = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type,
                          truncating=trunc_type)
    return padded

prep = preprocessText(df.News_Title)

In [12]:
prep = model.predict(prep)

In [13]:
prep

array([[1.22218654e-01, 4.97574478e-01, 3.80206853e-01],
       [7.55074024e-02, 8.90025437e-01, 3.44671756e-02],
       [1.45645306e-01, 5.78031778e-01, 2.76322871e-01],
       [1.29724117e-02, 9.21055019e-01, 6.59725517e-02],
       [7.22619370e-02, 6.83586240e-01, 2.44151801e-01],
       [2.83169914e-02, 9.17935729e-01, 5.37473261e-02],
       [3.01218070e-02, 9.62091088e-01, 7.78717222e-03],
       [1.48469001e-01, 3.41883123e-01, 5.09647846e-01],
       [1.08326949e-01, 4.80471849e-01, 4.11201209e-01],
       [2.19629422e-01, 6.17166162e-01, 1.63204312e-01],
       [6.27711117e-02, 7.91860342e-01, 1.45368531e-01],
       [6.97675705e-01, 3.19041088e-02, 2.70420164e-01],
       [3.95949893e-02, 9.16008115e-01, 4.43968847e-02],
       [8.35879892e-02, 8.66895139e-01, 4.95168604e-02],
       [3.08622816e-03, 5.11746407e-01, 4.85167354e-01],
       [4.94624972e-02, 9.13413227e-01, 3.71242464e-02],
       [2.89916158e-01, 5.14024556e-01, 1.96059272e-01],
       [3.73195880e-03, 3.40235

In [14]:
df['sent'] = np.argmax(prep, axis=-1)

In [15]:
df

,News_Title,Time,sent
0,"Is Nio Stock A Buy? Chip Shortage, EV Competit...",Mar-26-21 03:53PM,1
1,Column: Feds rule that Musk and Tesla are no f...,03:21PM,1
2,How the pandemic has led to a boom in used car...,03:08PM,1
3,"Dow, Tech Stocks Rally As Treasury Yields Jump...",01:19PM,1
4,Tesla's Touchscreen-Based Gear Selection Doesn...,01:11PM,1
...,...,...,...
95,Elon Musk Confirms Refreshed Tesla Model S Wil...,05:53PM,1
96,Tesla Supplier AU Optronics In Talks To Manufa...,03:48PM,1
97,Tesla's in-car cameras raise privacy concerns:...,02:39PM,1
98,"Dow, Tech Stocks Fall As Powell Calls U.S. Eco...",12:04PM,1


In [16]:
df.sent.value_counts()

1    85
2    11
0     4
Name: sent, dtype: int64